In [1]:
import numpy as np
import time
from utils import model
from inference import log_marginal_likelihood
from utils import hyperparameters

pi = 0.1
mu = 0.0
sigmabg = 0.001
sigma = 0.01
tau = 1 / (0.05 * 0.05)

x, y, csnps, v = model.simulate(pi = pi,
                             mu = mu,
                             sigma = sigma,
                             sigmabg = sigmabg,
                             tau = tau)

nvar = x.shape[0]
nsample = x.shape[1]
params = np.array([pi, mu, sigma, sigmabg, tau])

In [2]:
import os
import ctypes

def prune(newz, prob, oldprobsum, target):
    
    ''' Probability is for the new newz. 
        sum(prob) = 1 - oldprobsum    
    '''
    
    znorm = len(newz[0])
    print ("Pruning z-states of znorm {:d}".format(znorm))
    sort = np.argsort(prob)[::-1]             # index of decreasing order of prob. prob[sort] should be the sorted array
    cum  = oldprobsum + np.cumsum(prob[sort]) # cumulative sum, ensured that it will reach 1
    nsel = np.where(cum > target)[0]          # find where cum > target

    # assure there is at least one zstate 
    # sometimes posterior values could be so low that they can round off to zero
    # and there would be no state with cum > targ
    if len(nsel) == 0:
        leadk = [[]]
    else:
        zlen = nsel[0] + 1                # when cum > targ for first time (add +1 since indexing starts from 0 / if nsel[0] = 4, then there are 5 states with higher probabilities)
        sel  = sort[:zlen]                # These are our new selection
        sel  = np.sort(sel)               # How about sorting them?
        zlen = len(sel)

        # For debug
        #print(probsum)
        #print(prob[sort][:10] / probsum)

        # These are our leading states from which terms with znorm (k+1) will be created
        leadk = [newz[sel[i]] for i in range(zlen)]
        
    return leadk

def create(scaledparams, x, y, cmax, nvar, target):

    #_path = os.path.dirname(__file__)
    _path = os.getcwd()
    lib = np.ctypeslib.load_library('lib/zstates.so', _path)
    zcreate = lib.create_zstates
    zcreate.restype  = ctypes.c_int
    zcreate.argtypes = [ctypes.c_int,
                        ctypes.c_int,
                        ctypes.c_int,
                        np.ctypeslib.ndpointer(ctypes.c_int, flags='C_CONTIGUOUS, ALIGNED'),
                        np.ctypeslib.ndpointer(ctypes.c_int, flags='C_CONTIGUOUS, ALIGNED')]

    # Initialize for ||z|| = 0 and 1
    zstates = [[]]
    oldk = zstates
    newk = [[i] for i in range(nvar)]

    # Calculate the posterior probability of the zstates
    posterior   = log_marginal_likelihood.iterative_inverse(scaledparams, x, y, oldk + newk, grad = False)[2]
    prob        = np.array(posterior[-len(newk):])
    old_prob    = np.array(posterior[:len(oldk)])
    probsum     = np.sum(prob)
    old_probsum = np.sum(old_prob)

    # Add the ones required
    selk = prune(newk, prob, old_probsum, target)
    if len(selk) > 0:
        zstates += selk

    oldk = selk

    # Iterate over ||z|| = 2 to cmax
    norm = 1
    while norm < cmax:

        # Stop iteration if sum(new posterior) is < 0.02 times sum(old posterior)
        if probsum < (1 - target) * old_probsum:
            break

        norm += 1
        nsel = len(selk)

        # assure there is at least one zstate 
        # sometimes posterior values could be so low that they can round off to zero
        # and there would be no state with cum > targ
        if nsel == 0:
            break;
        else:
            leadk = np.array(selk, dtype=np.int32).reshape(nsel * (norm-1))

            # for first lead create all possible combinations
            # from next lead onwards do not include duplicate combinations.
            #    Note that a duplicate (k+1) entry is possible iff 
            #    two k-mers have at least (k-1) similar elements.
            #    e.g. [2,4,5,8] can be obtained from [2,4,5], [2,4,8], [2,5,8] or [4,5,8]
            # check previous leads to see if any of them has (k-1) elements similar to the current one
            # If so discard the duplicate.
            #
            # ^^^ the above logic has now been moved to a C++ function for speed up. 
            # get the new zstates from a C++ function
            maxnewsize = nsel * (nvar - norm + 1) * norm
            newz       = np.zeros(maxnewsize, dtype=np.int32)
            newstates  = zcreate(nsel, norm-1, nvar, leadk, newz)
            newsize    = newstates * norm
            result     = np.array(newz[:newsize]).reshape((newstates, norm))
            newk       = [sorted(list(result[i])) for i in range(newstates)]

            posterior   = log_marginal_likelihood.iterative_inverse(scaledparams, x, y, oldk + newk, grad = False)[2]
            prob        = np.array(posterior[-len(newk):])
            old_prob    = np.array(posterior[:len(oldk)])
            probsum     = np.sum(prob)
            old_probsum = np.sum(old_prob)

            # Add the ones required
            selk = prune(newk, prob, old_probsum, target)
            if len(selk) > 0:
                zstates += selk
                #print(selk)
                
            oldk = selk

    return zstates

In [3]:
cmax = 4
target = 0.9
scaledparams = hyperparameters.scale(params)

In [4]:
zstates = create(scaledparams, x, y, cmax, nvar, target)

Pruning z-states of znorm 1
Pruning z-states of znorm 2
Pruning z-states of znorm 3
Pruning z-states of znorm 4


In [6]:
zstates

[[],
 [58],
 [58, 151],
 [0, 58, 151],
 [4, 58, 151],
 [6, 58, 151],
 [7, 58, 151],
 [9, 58, 151],
 [10, 58, 151],
 [12, 58, 151],
 [14, 58, 151],
 [16, 58, 151],
 [19, 58, 151],
 [21, 58, 151],
 [26, 58, 151],
 [27, 58, 151],
 [34, 58, 151],
 [46, 58, 151],
 [48, 58, 151],
 [55, 58, 151],
 [57, 58, 151],
 [58, 63, 151],
 [58, 65, 151],
 [58, 67, 151],
 [58, 69, 151],
 [58, 70, 151],
 [58, 89, 151],
 [58, 94, 151],
 [58, 96, 151],
 [58, 102, 151],
 [58, 114, 151],
 [58, 116, 151],
 [58, 118, 151],
 [58, 128, 151],
 [58, 144, 151],
 [58, 145, 151],
 [58, 151, 154],
 [58, 151, 156],
 [58, 151, 164],
 [58, 151, 173],
 [58, 151, 174],
 [58, 151, 175],
 [58, 151, 176],
 [58, 151, 180],
 [58, 151, 181],
 [58, 151, 182],
 [58, 151, 185],
 [58, 151, 187],
 [58, 151, 189],
 [58, 151, 192],
 [58, 151, 194],
 [58, 151, 199],
 [0, 58, 102, 151],
 [0, 58, 151, 174],
 [4, 58, 102, 151],
 [4, 58, 151, 174],
 [6, 16, 58, 151],
 [6, 55, 58, 151],
 [6, 58, 69, 151],
 [6, 58, 102, 151],
 [6, 58, 151, 174

In [7]:
csnps[np.argsort(csnps)]

array([  6,  16,  34,  36,  55,  58,  63,  69,  98, 102, 114, 133, 134,
       150, 151, 164, 174, 175, 178, 181, 192, 194])

In [9]:
prob = log_marginal_likelihood.iterative_inverse(scaledparams, x, y, zstates, grad = False)[2]
sort = np.argsort(prob)[::-1]         # index of decreasing order of prob. prob[sort] should be the sorted array
cum  = np.cumsum(prob[sort])          # cumulative sum -- should be greater than target value
cum

array([ 0.20866717,  0.32879855,  0.40600277,  0.45944653,  0.49983506,
        0.52945849,  0.55303533,  0.56654414,  0.57949904,  0.58671964,
        0.59242587,  0.59734201,  0.60220937,  0.60694299,  0.61079531,
        0.61438904,  0.61783926,  0.6212691 ,  0.62458773,  0.6278833 ,
        0.63103161,  0.63404581,  0.63696802,  0.63981568,  0.64231025,
        0.64472451,  0.64708315,  0.64939323,  0.65167927,  0.65388461,
        0.65598561,  0.65806912,  0.66012447,  0.66206487,  0.66400514,
        0.66588605,  0.66773004,  0.66952796,  0.67132298,  0.67310215,
        0.67487726,  0.67661145,  0.67831816,  0.68001989,  0.68170137,
        0.68338046,  0.68503768,  0.68666345,  0.68827422,  0.68985546,
        0.69143281,  0.69296925,  0.69449106,  0.69601205,  0.69752698,
        0.6990297 ,  0.70051402,  0.70198562,  0.70345427,  0.70492172,
        0.70638665,  0.70784103,  0.70928662,  0.71071539,  0.71214071,
        0.71355095,  0.71496037,  0.71633219,  0.71769731,  0.71

In [10]:
prob

array([  4.08010272e-12,   7.56555978e-07,   1.41023945e-03,
         6.92372008e-05,   6.57158074e-05,   3.31863892e-03,
         6.57133218e-05,   7.40639178e-05,   7.59452392e-05,
         7.76064345e-05,   6.75682047e-05,   1.79792128e-03,
         7.08434846e-05,   1.10174760e-04,   8.85848108e-05,
         7.89312726e-05,   2.42322695e-04,   1.15842494e-04,
         1.14986891e-04,   9.13586130e-04,   1.33384501e-04,
         1.17366912e-04,   7.89655443e-05,   8.18268734e-05,
         3.32758114e-04,   6.82825416e-05,   7.98903810e-05,
         6.98429783e-05,   1.04164743e-04,   2.35768426e-02,
         6.11586617e-05,   6.94902925e-05,   8.82406382e-05,
         7.63362647e-05,   1.34971915e-04,   7.84761520e-05,
         7.75642854e-05,   7.57498341e-05,   1.10831810e-04,
         6.80687969e-05,   2.96234257e-02,   1.34166513e-04,
         6.81433829e-05,   1.31786888e-04,   4.44609147e-04,
         1.11640280e-04,   6.34570230e-05,   6.45888721e-05,
         6.24271043e-05,

In [25]:
from inference import zstates_old_method
from inference import zstates
import time
%load_ext snakeviz

cmax = 4
target = 0.99

start_time = time.time()
zstates_old = zstates_old_method.create(scaledparams, x, y, cmax, nvar, target)
prob = log_marginal_likelihood.iterative_inverse(scaledparams, x, y, zstates_old, grad = False)[2]
print ("zstates with old method created in {:g} seconds".format(time.time() - start_time))

#start_time = time.time()
#% snakeviz zstates_new = zstates.create(scaledparams, x, y, cmax, nvar, target)
#print ("zstates with new method created in {:g} seconds".format(time.time() - start_time))

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz
zstates with old method created in 46.3871 seconds


In [17]:
len(zstates_old)

29144

In [18]:
len(zstates_new)

14574